In [ ]:
!unzip /content/drive/MyDrive/dataset.zip

In [1]:
import numpy as np
import pandas as pd
import librosa
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import IPython.display as ipd
from tensorflow.keras.optimizers import SGD
from keras import regularizers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
l1 = os.listdir('/content/X')
filename_data = pd.DataFrame(l1 , columns = ['filename'])
l1.sort()
l2 = os.listdir('/content/Y')
event_data= pd.DataFrame(l2 , columns = ['eventname'])
l2.sort()


In [5]:
def data_loader(data,labels):
  d=[]
  l=[]
  for i in range (11):
    j=0
    for z in range (1000):
      if (labels[i][z]!=1 or z-j==200):
        if z-j>=200:
          for t in range (1):
            p=np.zeros((64))
            for tt in range (64):
              s=0
              for ttt in range(j,z):
                s+=data[t][tt][ttt]
              p[tt]=s/200
          d.append(p)
          l.append(i)
        j=z
  return d,l


In [6]:
data=[]
labels=[]
s=0
p=0
for row1,row2 in zip(l1,l2):
  
  file_name1 = os.path.join(os.path.abspath('/content/X'),str(row1))
  file_name2 = os.path.join(os.path.abspath('/content/Y'),str(row2))
  x,y=data_loader(np.load(file_name1),np.load(file_name2))

  for a in x:
    
    data.append(a)
  for a in y:
    labels.append(a)
  p+=len(x)
  for a in x:
    s+=len(a)


In [7]:
print(s/p,s,p)
len(data)

64.0 2247872 35123


35123

In [8]:
def r_shape(data):
  data = np.reshape(data,(35123,16,4))
  data = np.reshape(data,(35123,16,4,1))
  return data

data=r_shape(data)

y=np.array(pd.get_dummies(labels))

In [9]:
x_train, x_test, y_train, y_test = train_test_split(data,y, test_size=0.10,random_state=42)

In [10]:
input_shape=(16,4,1)
x_train.shape,y_train.shape


((31610, 16, 4, 1), (31610, 11))

In [11]:
model =  models.Sequential([
    
                          layers.Conv2D(64 , (2,2),activation = 'relu',padding='valid', input_shape = input_shape),  
                          layers.MaxPooling2D(2, padding='same'),
                          layers.Conv2D(128, (2,2), activation='relu',padding='valid'),
                          layers.MaxPooling2D(2, padding='same'),
                          layers.Flatten(),
                          layers.Dense(64 , activation = 'relu'),
                          layers.Dense(11 , activation = 'softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = 'acc')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 15, 3, 64)         320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 2, 64)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 1, 128)         32896     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 1, 128)        0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 64)                3

In [12]:
history = model.fit(x_train, y_train, epochs=20, validation_data= (x_test, y_test),batch_size=32)

Epoch 1/20
988/988 [==============================] - 15s 4ms/step - loss: 1.4024 - acc: 0.5049 - val_loss: 1.2487 - val_acc: 0.5574
Epoch 2/20
988/988 [==============================] - 4s 4ms/step - loss: 1.2403 - acc: 0.5557 - val_loss: 1.2046 - val_acc: 0.5727
Epoch 3/20
988/988 [==============================] - 4s 4ms/step - loss: 1.1941 - acc: 0.5711 - val_loss: 1.1503 - val_acc: 0.5915
Epoch 4/20
988/988 [==============================] - 4s 4ms/step - loss: 1.1472 - acc: 0.5931 - val_loss: 1.1084 - val_acc: 0.6143
Epoch 5/20
988/988 [==============================] - 3s 4ms/step - loss: 1.1026 - acc: 0.6125 - val_loss: 1.0555 - val_acc: 0.6436
Epoch 6/20
988/988 [==============================] - 3s 3ms/step - loss: 1.0690 - acc: 0.6252 - val_loss: 1.0304 - val_acc: 0.6482
Epoch 7/20
988/988 [==============================] - 4s 4ms/step - loss: 1.0391 - acc: 0.6345 - val_loss: 1.0080 - val_acc: 0.6584
Epoch 8/20
988/988 [==============================] - 4s 4ms/step - loss: 1

In [13]:
def eventroll_to_multihot_vector(eventroll):
    """
    Parameters
    ----------
    eventroll : np.array
        Eventroll matrix of shape=(11, 1000).
    
    Returns
    -------
    np.array
        A multihot vector of shape=(10,)
    """
    
    # findout active events:
    active_events = (eventroll.sum(axis=1) >= 0.5).astype('float')
    
    # remove silence class:
    return np.array(active_events)

In [14]:

def r_shape_2(data):
  data = np.reshape(data,(1,16,4))
  data = np.reshape(data,(1,16,4,1))
  return data


def prediction(model,test_data,windowsize):
  arr=[]
  for a in range(1):
    p=np.zeros((64))
    for b in range(64):
      s=0
      for c in range(0,1000-windowsize,windowsize):
        for k in range (c,c+windowsize):
          s+=test_data[a][b][k]
      p[b]=s/windowsize
    arr.append(p)
  arr=np.array(arr)
  y=np.zeros((11))
  for a in arr:
    ll=r_shape_2(a)
    #print(ll.shape)
    x=model.predict(ll)
    x=x.flatten()
    #print(x.shape)
    k=-100000
    for i in  range(11):
      k=max(k,x[i])
    for i in range (11):
      if x[i]==k:
        y[i]=1
  return y
  


In [15]:
data=np.zeros((10000,1,64,1000))
labels=np.zeros((10000,11))
def r_shape_1(data):
  data = np.reshape(data,(64,1000))
  data = np.reshape(data,(64,1000,1))
  return data

for i,row in filename_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/X'),str(row["filename"]))
  d=np.load(file_name)
  data[i]=d
for i,row in event_data.iterrows():
  file_name = os.path.join(os.path.abspath('/content/Y'),str(row["eventname"]))
  e = np.load(file_name)
  e=eventroll_to_multihot_vector(e)
  labels[i]=e

In [16]:
y_pred=np.zeros((10000,11))
for i in range(10000):
  y_pred[i]=prediction(model,data[i],200)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step


In [17]:
from sklearn.metrics import f1_score
k=f1_score(labels, y_pred, average=None)
print(k)
k=np.mean(k,axis=0)
print(k)

[0.03229665 0.04570528 0.06257822 0.00081934 0.03018868 0.10167866
 0.11990408 0.09295904 0.19707842 0.48573484 0.11280362]
0.11652243867464174
